In [24]:
import pyspark
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import mean_absolute_error
# from sklearn import tree
# from sklearn.metrics import accuracy_score

from pyspark.sql import SparkSession
spark=SparkSession.builder.appName("CS504_Project").getOrCreate()
from pyspark.sql.functions import *
from datetime import date, datetime
import time
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

DataFrame[key: string, value: string]

In [2]:
####################################################################################################################################################################
# Read all files
####################################################################################################################################################################
# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","
file_type_csv = "csv"


country_distance = "data/country_distance.csv"

country_distance_df = spark.read.format(file_type_csv) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(country_distance)
country_distance_df.createOrReplaceTempView("COUNTRY_DISTANCE")

# War Events File.
war_events = "data/war_events.csv"

war = spark.read.format(file_type_csv) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(war_events)

# display(war)

# war.createOrReplaceTempView("WAR_EVENTS")

# Commodity Export File.
commodity_exports = "data/commodity_exports.csv"

exports_raw = spark.read.format(file_type_csv) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(commodity_exports)

exports = exports_raw.withColumn("Commodity", when(exports_raw.Commodity == "Dog or cat food; (not put up for retail sale), used in animal feeding","AnimalFodder") \
      .when(exports_raw.Commodity == "Wheat and meslin","Wheat") \
      .when(exports_raw.Commodity == "Iron ores and concentrates; including roasted iron pyrites","IronOre") \
      .when(exports_raw.Commodity == "Slag, dross; (other than granulated slag), scalings and other waste from the manufacture of iron or steel","Slag") \
      .when(exports_raw.Commodity == "Base metals, silver or gold, clad with platinum; not further worked than semi-manufactured","GoldOre") \
      .when(exports_raw.Commodity == "Cereal groats and meal; of maize (corn)","Corn") \
     .when(exports_raw.Commodity == "Vegetable oils; sunflower seed or safflower oil and their fractions, other than crude, whether or not refined, but not chemically modified","SunflowerOil") \
     .when(exports_raw.Commodity == "Vegetable oils; low erucic acid rape or colza oil and its fractions, crude","RapeseedOil") \
     .when(exports_raw.Commodity == "Copper ores and concentrates","Copper") \
     .when(exports_raw.Commodity == "Coal; anthracite, whether or not pulverised, but not agglomerated","Coal") \
     .when(exports_raw.Commodity == "Oils; petroleum oils and oils obtained from bituminous minerals, crude","CrudePetroleum") \
     .when(exports_raw.Commodity == "Petroleum gases and other gaseous hydrocarbons; liquefied, natural gas","RefinedPetroleum") \
     .when(exports_raw.Commodity == "Iron or steel, pig iron, spiegeleisen; granules thereof","Steel") \
     .when(exports_raw.Commodity == "Steel, stainless; cold-drawn or cold-rolled, tubes and pipes of circular cross-section","RolledSteel") \
     .when(exports_raw.Commodity == "Cooking appliances and plate warmers; for gas fuel or for both gas and other fuels, of iron or steel","Gas") \
     .when(exports_raw.Commodity == "Steel, stainless; table, kitchen and other household articles and parts thereof","StainlessSteel") \
     .when(exports_raw.Commodity == "Iron or steel; cast articles, excluding grinding balls and similar articles for mills, other than of non-malleable cast iron","CastIron") \
     .when(exports_raw.Commodity == "Nickel; unwrought, not alloyed","Nickel") \
     .when(exports_raw.Commodity == "Aluminium; unwrought, alloys","Aluminium") \
     .when(exports_raw.Commodity == "Barley","Barley") \
      )


exports=exports.withColumnRenamed("Netweight (kg)","NETWEIGHT").withColumnRenamed("Trade Value (US$)", "TRADEVALUEINUSD")
# display(exports)
exports.createOrReplaceTempView("EXPORTS")

# Commodity Prices File.
commodity_prices = "data/commodity_prices.csv"

prices = spark.read.format(file_type_csv) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(commodity_prices)


prices=prices.withColumnRenamed("Date","Starting_Date")
# display(prices)
prices.createOrReplaceTempView("PRICES")

22/11/12 13:33:34 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [3]:
####################################################################################################################################################################
# Calculating the Year-Month for all Files
####################################################################################################################################################################
transform_war_events=war.withColumn("EVENT_YEAR_MONTH", date_format(to_date(concat(col("YEAR"), col("EVENT_MONTH")),"yyyyMMM"),"yyyyMM"))
group_war_events=transform_war_events.groupBy("EVENT_YEAR_MONTH").agg(count("EVENT_TYPE").alias("COUNT_OF_EVENTS"))

# group_war_events.show()
group_war_events.createOrReplaceTempView("GROUP_EVENTS")

transform_exports=spark.sql(""" 
                               select  *,date_format(TO_DATE(Period,'yyyyMM'),'yyyyMM') as PERIOD_YEAR_MONTH from EXPORTS
                              """)
# transform_exports.show(3)
transform_exports.createOrReplaceTempView("GROUP_EXPORTS")

transform_prices=spark.sql(""" 
                               select  *,date_format(TO_DATE(Starting_Date,'MM/dd/yyyy'),'yyyyMM') as DATE_YEAR_MONTH from PRICES
                              """)
# transform_prices.show(3)
transform_prices.createOrReplaceTempView("GROUP_PRICES")

In [4]:
####################################################################################################################################################################
# Joining of all Files Based on Date
####################################################################################################################################################################
join_events_prices_exports=spark.sql(""" 
                               select  EVENT_YEAR_MONTH, 
                                       COUNT_OF_EVENTS,
                                       Reporter as REPORTER,
                                       Partner as PARTNER,
                                       GROUP_EXPORTS.Commodity,
                                       CASE
                                           WHEN INT(NETWEIGHT) is NULL THEN 1
                                           ELSE INT(NETWEIGHT)
                                       END AS NETWEIGHT,
                                       CASE
                                           WHEN INT(TRADEVALUEINUSD) is NULL  THEN 1
                                           ELSE INT(TRADEVALUEINUSD)
                                       END AS TRADEVALUEINUSD,
                                       
                                       CASE
                                           WHEN INT(NETWEIGHT) is NULL  AND INT(TRADEVALUEINUSD) is NULL  THEN 1
                                           WHEN INT(NETWEIGHT) is NULL                                    THEN INT(TRADEVALUEINUSD)
                                           WHEN INT(TRADEVALUEINUSD) is NULL                              THEN INT(NETWEIGHT)
                                           ELSE DOUBLE(NETWEIGHT) * DOUBLE(TRADEVALUEINUSD)
                                       END AS TOTAL_SOLD_USD,
                                       DOUBLE(Amount) as PRODUCERPRICE,
                                       Case
                                           When Partner = 'Ukraine' then COUNTRY_DISTANCE.Ukraine
                                           When Partner = 'Russian Federation' then COUNTRY_DISTANCE.Russia
                                       End as DISTANCE_COUNTRIES
                                       from 
                                       GROUP_EVENTS,
                                       GROUP_PRICES,
                                       GROUP_EXPORTS,
                                       COUNTRY_DISTANCE
                                       where
                                           EVENT_YEAR_MONTH = PERIOD_YEAR_MONTH
                                       AND PERIOD_YEAR_MONTH = DATE_YEAR_MONTH
                                       AND GROUP_EXPORTS.Commodity = GROUP_PRICES.Commodity
                                       AND COUNTRY_DISTANCE.Country = Reporter
                                     
                              """)
join_events_prices_exports.show()

+----------------+---------------+------------------+------------------+---------+---------+---------------+----------------+-------------+------------------+
|EVENT_YEAR_MONTH|COUNT_OF_EVENTS|          REPORTER|           PARTNER|Commodity|NETWEIGHT|TRADEVALUEINUSD|  TOTAL_SOLD_USD|PRODUCERPRICE|DISTANCE_COUNTRIES|
+----------------+---------------+------------------+------------------+---------+---------+---------------+----------------+-------------+------------------+
|          201809|           1363|            France|           Ukraine|    Wheat|  1290293|         334605|4.31738489265E11|  191.1444218|              2687|
|          201809|           1363|         Lithuania|Russian Federation|    Wheat|    20000|           6997|        1.3994E8|  191.1444218|              4484|
|          201809|           1363|           Finland|Russian Federation|    Steel|        1|             13|            13.0|        245.2|              3922|
|          201809|           1363|           H

In [5]:
join_events_prices_exports.coalesce(1).write.format("parquet").mode("overwrite").save("data/Joined_Events_Prices_Exports.parquet")

In [6]:
join_events_prices_exports.coalesce(1).write.format("csv").mode("overwrite").option("sep",",").option("header",True).save("data/Joined_Events_Prices_Exports.csv")

In [7]:
# join_events_prices_exports.show(4)

+----------------+---------------+---------+------------------+---------+---------+---------------+----------------+-------------+------------------+
|EVENT_YEAR_MONTH|COUNT_OF_EVENTS| REPORTER|           PARTNER|Commodity|NETWEIGHT|TRADEVALUEINUSD|  TOTAL_SOLD_USD|PRODUCERPRICE|DISTANCE_COUNTRIES|
+----------------+---------------+---------+------------------+---------+---------+---------------+----------------+-------------+------------------+
|          201809|           1363|   France|           Ukraine|    Wheat|  1290293|         334605|4.31738489265E11|  191.1444218|              2687|
|          201809|           1363|Lithuania|Russian Federation|    Wheat|    20000|           6997|        1.3994E8|  191.1444218|              4484|
|          201809|           1363|  Finland|Russian Federation|    Steel|        1|             13|            13.0|        245.2|              3922|
|          201809|           1363|  Hungary|Russian Federation|    Wheat|    20000|           3935| 

In [8]:

join_events_prices_exports=pd.read_parquet('data/Joined_Events_Prices_Exports.parquet', engine='pyarrow')
nRow, nCol = join_events_prices_exports.shape
print(f'There are {nRow} rows and {nCol} columns\n\n')
join_events_prices_exports.head(5)

There are 11477 rows and 10 columns




,EVENT_YEAR_MONTH,COUNT_OF_EVENTS,REPORTER,PARTNER,Commodity,NETWEIGHT,TRADEVALUEINUSD,TOTAL_SOLD_USD,PRODUCERPRICE,DISTANCE_COUNTRIES
0,201809,1363,France,Ukraine,Wheat,1290293,334605,4.317385e+11,191.144422,2687
1,201809,1363,Lithuania,Russian Federation,Wheat,20000,6997,1.399400e+08,191.144422,4484
2,201809,1363,Finland,Russian Federation,Steel,1,13,1.300000e+01,245.200000,3922
3,201809,1363,Hungary,Russian Federation,Wheat,20000,3935,7.870000e+07,191.144422,5342
4,201809,1363,Hungary,Ukraine,Wheat,17685,5261,9.304078e+07,191.144422,1194


In [9]:
join_events_prices_exports.info()

join_events_prices_exports.isnull().sum()
print("Total missing values:",join_events_prices_exports.isna().sum().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11477 entries, 0 to 11476
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   EVENT_YEAR_MONTH    11477 non-null  object 
 1   COUNT_OF_EVENTS     11477 non-null  int64  
 2   REPORTER            11477 non-null  object 
 3   PARTNER             11477 non-null  object 
 4   Commodity           11477 non-null  object 
 5   NETWEIGHT           11477 non-null  int32  
 6   TRADEVALUEINUSD     11477 non-null  int32  
 7   TOTAL_SOLD_USD      11477 non-null  float64
 8   PRODUCERPRICE       11477 non-null  float64
 9   DISTANCE_COUNTRIES  11477 non-null  object 
dtypes: float64(2), int32(2), int64(1), object(5)
memory usage: 807.1+ KB
Total missing values: 0


In [10]:
numerical_cols=join_events_prices_exports.select_dtypes(['int64','float64','int32'])
categorical_cols = join_events_prices_exports.select_dtypes('object').columns
print("There are ",len(numerical_cols.columns), " numerical features and they are : \n", numerical_cols.columns)
print("There are ",len(categorical_cols), " categorical features and they are : \n", categorical_cols)

There are  5  numerical features and they are : 
 Index(['COUNT_OF_EVENTS', 'NETWEIGHT', 'TRADEVALUEINUSD', 'TOTAL_SOLD_USD',
       'PRODUCERPRICE'],
      dtype='object')
There are  5  categorical features and they are : 
 Index(['EVENT_YEAR_MONTH', 'REPORTER', 'PARTNER', 'Commodity',
       'DISTANCE_COUNTRIES'],
      dtype='object')


In [11]:
# print(join_events_prices_exports)

      EVENT_YEAR_MONTH  COUNT_OF_EVENTS   REPORTER             PARTNER  \
0               201809             1363     France             Ukraine   
1               201809             1363  Lithuania  Russian Federation   
2               201809             1363    Finland  Russian Federation   
3               201809             1363    Hungary  Russian Federation   
4               201809             1363    Hungary             Ukraine   
...                ...              ...        ...                 ...   
11472           202106              588    Belarus             Ukraine   
11473           202106              588      India             Ukraine   
11474           202106              588      India             Ukraine   
11475           202106              588         EU             Ukraine   
11476           202106              588         EU             Ukraine   

            Commodity  NETWEIGHT  TRADEVALUEINUSD  TOTAL_SOLD_USD  \
0               Wheat    1290293          

In [16]:


commodity_list=join_events_prices_exports['Commodity'].astype(str).unique().tolist()
PARTNER_list=join_events_prices_exports['PARTNER'].astype(str).unique().tolist()

df_list=[]
for partner in PARTNER_list:
    partnerval=partner.replace(' ','')
    for commodity in commodity_list:
        dynamic_df_Name = "df_"+partnerval+"_commodity_"+commodity
        df_list.append(dynamic_df_Name)
        # locals()[dynamic_df_Name]=join_events_prices_exports[(join_events_prices_exports['PARTNER'] == partner) & (join_events_prices_exports['Commodity'] == commodity)]
        locals()[dynamic_df_Name]=join_events_prices_exports[(join_events_prices_exports['PARTNER'] == partner) & (join_events_prices_exports['Commodity'] == commodity)][["EVENT_YEAR_MONTH","COUNT_OF_EVENTS","TOTAL_SOLD_USD","PRODUCERPRICE","DISTANCE_COUNTRIES"]]


In [17]:
models_comparison_matrix = pd.DataFrame([],columns=['Regression_Commodity_Partner','Predictive_Analysis_Model', 'Accuracy_Score','Mean_Absolute_Error', 'Root_Mean_Squared_Error'])

In [28]:
######################################################################################################################
#                                             Linear Regression                                                      #
#--------------------------------------------------------------------------------------------------------------------#
# Using features :                                #
######################################################################################################################

linear_regression_model=LinearRegression()

for df_name in df_list:  
    print(df_name.split("_")[1] + "_" + df_name.split("_")[3])
    if locals()[df_name].shape[0] > 1:
        X=locals()[df_name].values
        Y=locals()[df_name]['TOTAL_SOLD_USD']
        x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2,random_state=0)
        clf=linear_regression_model.fit(x_train, y_train)
        # print("Coefficient/Slope : ", clf.coef_)
        # print("Intercept : ", clf.intercept_)
        linear_regression_predictions = linear_regression_model.predict(x_test)
        clf.score(x_test,y_test)
    
        linear_regression_model_mae=mean_absolute_error(y_test, linear_regression_predictions)
        linear_regression_model_rmse=np.sqrt(metrics.mean_squared_error(y_test, linear_regression_predictions))
        linear_regression_model_score=clf.score(x_test,y_test)
    
        # print("\n\nMean Absolute Error:",linear_regression_model_mae)
        # print("RMSE: ",linear_regression_model_rmse)
        # print("Accuracy score of the model : ", linear_regression_model_score)
        models_comparison_matrix = models_comparison_matrix.append({
            'Regression_Commodity_Partner': df_name.split("_")[1] + "_" + df_name.split("_")[3], 
            'Predictive_Analysis_Model': linear_regression_model.__class__.__name__,
            'Accuracy_Score': linear_regression_model_score,
            'Mean_Absolute_Error' : linear_regression_model_mae,
            'Root_Mean_Squared_Error':linear_regression_model_rmse
        }, ignore_index=True)

Ukraine_Wheat


Mean Absolute Error: 0.0030839387748642366
RMSE:  0.003803031931430593
Accuracy score of the model :  1.0
Ukraine_Steel


Mean Absolute Error: 2.0065782862843662e-07
RMSE:  2.332595610993155e-07
Accuracy score of the model :  1.0
Ukraine_Aluminium


Mean Absolute Error: 6.465809075256848e-05
RMSE:  0.00012413728828775548
Accuracy score of the model :  1.0
Ukraine_StainlessSteel


Mean Absolute Error: 8.764278864124752e-05
RMSE:  0.00016036954105227928
Accuracy score of the model :  1.0
Ukraine_Gas


Mean Absolute Error: 2.451194565145208e-05
RMSE:  3.8155056646223766e-05
Accuracy score of the model :  1.0
Ukraine_CastIron


Mean Absolute Error: 4.346202119158761e-06
RMSE:  6.973610077046577e-06
Accuracy score of the model :  1.0
Ukraine_RapeseedOil


Mean Absolute Error: 0.00017666069726171178
RMSE:  0.00027347013509296795
Accuracy score of the model :  1.0
Ukraine_Barley


Mean Absolute Error: 0.0006838224808706266
RMSE:  0.0034719588665192208
Accuracy score of the mod

/var/folders/f2/hsjy4hyj5ng0q9dlrpfcn0ph0000gn/T/ipykernel_8645/2794306911.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  models_comparison_matrix = models_comparison_matrix.append({
/var/folders/f2/hsjy4hyj5ng0q9dlrpfcn0ph0000gn/T/ipykernel_8645/2794306911.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  models_comparison_matrix = models_comparison_matrix.append({
/var/folders/f2/hsjy4hyj5ng0q9dlrpfcn0ph0000gn/T/ipykernel_8645/2794306911.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  models_comparison_matrix = models_comparison_matrix.append({
/var/folders/f2/hsjy4hyj5ng0q9dlrpfcn0ph0000gn/T/ipykernel_8645/2794306911.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas

RussianFederation_CastIron


Mean Absolute Error: 3.675994843954461e-05
RMSE:  5.3371605992902296e-05
Accuracy score of the model :  1.0
RussianFederation_RapeseedOil


Mean Absolute Error: 0.04044207935715865
RMSE:  0.056290902241377914
Accuracy score of the model :  1.0
RussianFederation_Barley


Mean Absolute Error: 0.0008493296775045307
RMSE:  0.0020205794313460155
Accuracy score of the model :  1.0
RussianFederation_SunflowerOil


Mean Absolute Error: 8.117117394943676e-05
RMSE:  0.00011810062647291601
Accuracy score of the model :  1.0
RussianFederation_IronOre


Mean Absolute Error: 2.2406216662890435
RMSE:  3.017264289609587
Accuracy score of the model :  1.0
RussianFederation_Copper


Mean Absolute Error: 0.2809552516877235
RMSE:  0.33412642262958264
Accuracy score of the model :  1.0
RussianFederation_CrudePetroleum


Mean Absolute Error: 0.006624142693189672
RMSE:  0.007930910997323539
Accuracy score of the model :  1.0
RussianFederation_Coal


Mean Absolute Error: 0.0134670

/var/folders/f2/hsjy4hyj5ng0q9dlrpfcn0ph0000gn/T/ipykernel_8645/2794306911.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  models_comparison_matrix = models_comparison_matrix.append({
/var/folders/f2/hsjy4hyj5ng0q9dlrpfcn0ph0000gn/T/ipykernel_8645/2794306911.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  models_comparison_matrix = models_comparison_matrix.append({
/var/folders/f2/hsjy4hyj5ng0q9dlrpfcn0ph0000gn/T/ipykernel_8645/2794306911.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  models_comparison_matrix = models_comparison_matrix.append({
/var/folders/f2/hsjy4hyj5ng0q9dlrpfcn0ph0000gn/T/ipykernel_8645/2794306911.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas

In [29]:
print(models_comparison_matrix)

          Regression_Commodity_Partner Predictive_Analysis_Model  \
0                        Ukraine_Wheat          LinearRegression   
1                        Ukraine_Steel          LinearRegression   
2                    Ukraine_Aluminium          LinearRegression   
3               Ukraine_StainlessSteel          LinearRegression   
4                          Ukraine_Gas          LinearRegression   
..                                 ...                       ...   
60              RussianFederation_Coal          LinearRegression   
61              RussianFederation_Slag          LinearRegression   
62              RussianFederation_Corn          LinearRegression   
63            RussianFederation_Nickel          LinearRegression   
64  RussianFederation_RefinedPetroleum          LinearRegression   

   Accuracy_Score Mean_Absolute_Error Root_Mean_Squared_Error  
0             1.0            0.003084                0.003803  
1             1.0                 0.0                  